# 📌 Projeto Pós-Graduação - Datathon Fase 5

## Objetivo
Construir um pipeline de Machine Learning para prever a aprovação ou reprovação de candidatos em processos seletivos, utilizando as bases fornecidas (`applicants.json`, `prospects.json`, `vagas.json`).  
O modelo será treinado com candidatos já aprovados/reprovados e aplicado nos candidatos que ainda estão "em andamento" no processo.

---

## Bases de Dados
1. **Applicants**  
   - Informações do candidato: dados pessoais, formação, experiências, currículos (`cv_pt`, `cv_en`).
   - Colunas aninhadas que precisam ser normalizadas (`infos_basicas`, `informacoes_pessoais`, `informacoes_profissionais`, etc.).

2. **Vagas**  
   - Detalhes das vagas abertas: título, modalidade, área, requisitos.
   - Contém também a lista de `prospects` vinculados a cada vaga.

3. **Prospects**  
   - Relaciona candidatos e vagas.
   - Coluna chave: `situacao_candidato` (status do processo seletivo).

---

## Situação do Target
A coluna `situacao_candidato` possui múltiplos status (ex.: *Aprovado, Reprovado, Em avaliação, Entrevista, Proposta Aceita*).  

- ~30% dos registros estão em **Aprovado/Reprovado** → servem de base para treino.  
- ~70% estão em **Em andamento** → onde aplicaremos o modelo.  

---

## Estratégia do Target
- **Aprovados (1):** `"Aprovado"`, `"Proposta Aceita"`, `"Contratado pela Decision"`.  
- **Reprovados (0):** `"Reprovado"`, `"Recusado"`, `"Não Aprovado pelo RH/Cliente/Requisitante"`, `"Desistiu"`, `"Sem interesse nesta vaga"`.  
- **Em andamento (NaN):** todos os demais status (ex.: `"Em avaliação pelo RH"`, `"Entrevista Técnica"`, `"Documentação CLT"`).

---

## Pipeline do Projeto

### 1. Pré-processamento
- Ler os três JSONs (`applicants`, `vagas`, `prospects`).
- Explodir e normalizar colunas aninhadas (listas/dicionários).
- Garantir identificadores únicos (`id_candidato`, `id_vaga`).

### 2. Integração
- Unir as bases:
  - `applicants + prospects` (via `id_candidato`).
  - Depois juntar com `vagas` (via `id_vaga`).
- Gerar o `df_final`.

### 3. Definição do Target
- Mapear `situacao_candidato` → `target`.
- Separar:
  - `df_train` → candidatos com target definido (aprovados/reprovados).
  - `df_predict` → candidatos em andamento (sem target).

### 4. Feature Engineering
- Texto: `cv_pt` → transformar com **TF-IDF**.
- Categóricas: `nivel_academico`, `area_atuacao`, etc. → OneHotEncoder.
- Numéricas: idade, tempo experiência (se disponível).

### 5. Modelagem
- Treinar modelos base: **Logistic Regression**, **RandomForest**.
- Comparar métricas (Acurácia, F1, Recall).
- Escolher o melhor e salvar em `.pkl`.

### 6. Aplicação
- Rodar o modelo no `df_predict`.
- Gerar probabilidade de aprovação para cada candidato em andamento.

### 7. Entrega Final
- Organizar repositório:


### Vagas

In [1]:
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:

with open("vagas.json", "r", encoding="utf-8") as f:
    jobs_raw = json.load(f)

df_jobs = pd.DataFrame.from_dict(jobs_raw, orient="index")
df_jobs.index.name = "id_vaga"

def flatten_sections(row):
    base   = {f"inf_{k}": v for k, v in row.get("informacoes_basicas", {}).items()}
    perfil = {f"perfil_{k}": v for k, v in row.get("perfil_vaga", {}).items()}
    benef  = {f"benef_{k}": v for k, v in row.get("beneficios", {}).items()}
    return {**base, **perfil, **benef}

flat_data = df_jobs.apply(flatten_sections, axis=1)
vagas = pd.DataFrame(list(flat_data), index=df_jobs.index).reset_index()

# Selecionar só as colunas úteis
colunas_util = [
    "id_vaga",
    "inf_titulo_vaga",
    "inf_cliente",
    "inf_vaga_sap",
    "perfil_nivel_academico",
    "perfil_nivel profissional",
    "perfil_nivel_ingles",
    "perfil_nivel_espanhol",
    "perfil_competencia_tecnicas_e_comportamentais",
    "perfil_principais_atividades"
]

df_vagas = vagas[colunas_util]


In [3]:
df_vagas.head()

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...
1,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...
2,5183,ANALISTA PL/JR C/ SQL,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Analista,Nenhum,Intermediário,Requisitos mandatórios:\n\no Conhecimentos Téc...,Descrição – Atividades:\n\no Monitoramento das...
3,5182,Technical Architect - 11894809,Nelson-Page,Não,Ensino Superior Completo,Analista,Básico,Básico,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...
4,5181,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Mann and Sons,Não,Ensino Superior Completo,Sênior,Intermediário,Nenhum,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...


## Prospect

In [4]:

# 1) Ler prospects.json
with open("prospects.json", "r", encoding="utf-8") as f:
    prospects_raw = json.load(f)

# 2) Criar DataFrame onde cada índice = id da vaga
df_prospects = pd.DataFrame.from_dict(prospects_raw, orient="index").reset_index()
df_prospects.rename(columns={"index": "id_vaga"}, inplace=True)

# 3) Explodir lista de candidatos (prospects) -> cada candidato vira uma linha
df_vagas_explodida = df_prospects.explode("prospects").reset_index(drop=True)

# 4) Normalizar o dicionário de cada candidato
df_prospects_expandido = pd.json_normalize(df_vagas_explodida["prospects"])

# 5) Concatenar: vaga + info do candidato
prospects = pd.concat(
    [df_vagas_explodida.drop(columns=["prospects"]).reset_index(drop=True),
     df_prospects_expandido.reset_index(drop=True)],
    axis=1
)

# 6) Padronizar colunas importantes
# Em muitos dumps o id do candidato aparece como "codigo"
if "codigo" in prospects.columns:
    prospects.rename(columns={"codigo": "id_candidato"}, inplace=True)

# Se status vier como "situacao" ou "status"
if "situacao" in prospects.columns:
    prospects.rename(columns={"situacao": "situacao_candidado"}, inplace=True)
elif "status" in prospects.columns:
    prospects.rename(columns={"status": "situacao_candidado"}, inplace=True)

# Garantir tipos numéricos
prospects["id_vaga"] = pd.to_numeric(prospects["id_vaga"], errors="coerce").astype("Int64")
prospects["id_candidato"] = pd.to_numeric(prospects["id_candidato"], errors="coerce").astype("Int64")

# 7) Manter só colunas úteis (ajuste se quiser mais info da vaga)
df_prospects = prospects[["id_vaga", "id_candidato", "nome","situacao_candidado", "data_candidatura", "recrutador", "titulo"]].dropna(subset=["id_candidato"])


In [5]:
df_prospects["situacao_candidado"].unique()

array(['Encaminhado ao Requisitante', 'Contratado pela Decision',
       'Desistiu', 'Documentação PJ', 'Não Aprovado pelo Cliente',
       'Prospect', 'Não Aprovado pelo RH', 'Aprovado',
       'Não Aprovado pelo Requisitante', 'Inscrito', 'Entrevista Técnica',
       'Em avaliação pelo RH', 'Contratado como Hunting',
       'Desistiu da Contratação', 'Entrevista com Cliente',
       'Documentação CLT', 'Recusado', 'Documentação Cooperado',
       'Sem interesse nesta vaga', 'Encaminhar Proposta',
       'Proposta Aceita'], dtype=object)

###  Etapa Importante — Definição do Target

Um ponto crucial do projeto é a definição da variável **target**.  
Ao analisar a base, identificamos que parte dos candidatos já possui status finais, como **“Aprovado”**, **“Contratado”** ou **“Reprovado/Recusado”**.  

Esses casos serão utilizados como **conjunto de treino**:  
- **1 → Aprovado/Contratado**  
- **0 → Reprovado/Recusado**

Com esse histórico, o modelo aprende os padrões e características que diferenciam candidatos aprovados dos reprovados.  
Assim, conseguimos **aplicar o modelo nos candidatos em andamento** (ex.: *“Em avaliação”, “Entrevista técnica”*), prevendo a probabilidade de aprovação de cada um.  

Dessa forma, os recrutadores passam a ter um **ranking de candidatos mais promissores**, o que permite investir tempo apenas nos perfis com maior chance de sucesso.


In [6]:
map_status = {
    # APROVADOS
    "Aprovado": 1,
    "Proposta Aceita": 1,
    "Contratado pela Decision": 1,

    # REPROVADOS
    "Reprovado": 0,
    "Recusado": 0,
    "Não Aprovado pelo RH": 0,
    "Não Aprovado pelo Cliente": 0,
    "Não Aprovado pelo Requisitante": 0,

    # desistência também pode ser considerado reprovação
    "Desistiu": 0,
    "Desistiu da Contratação": 0,
    "Sem interesse nesta vaga": 0
}

# Cria coluna target (1 = aprovado, 0 = reprovado, NaN = em andamento)
df_prospects["target"] = df_prospects["situacao_candidado"].map(map_status)

In [7]:
df_prospects["target"].value_counts(dropna=False)

,count
target,
NaN,41783
0.0,9008
1.0,2968


In [8]:
df_prospects.head()

,id_vaga,id_candidato,nome,situacao_candidado,data_candidatura,recrutador,titulo,target
0,4530,25632,José Vieira,Encaminhado ao Requisitante,25-03-2021,Ana Lívia Moreira,CONSULTOR CONTROL M,NaN
1,4530,25529,Srta. Isabela Cavalcante,Encaminhado ao Requisitante,22-03-2021,Ana Lívia Moreira,CONSULTOR CONTROL M,NaN
2,4531,25364,Sra. Yasmin Fernandes,Contratado pela Decision,17-03-2021,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Dom...,1.0
3,4531,25360,Alexia Barbosa,Encaminhado ao Requisitante,17-03-2021,Juliana Cassiano,2021-2607395-PeopleSoft Application Engine-Dom...,NaN
5,4533,26338,Arthur Almeida,Contratado pela Decision,29-04-2021,Stella Vieira,2021-2605708-Microfocus Application Life Cycle...,1.0


In [9]:
df_prospects = df_prospects[["id_vaga", "titulo", "id_candidato", "nome", "data_candidatura", "recrutador", "situacao_candidado", "target"]]

In [10]:
df_prospects.isnull().sum()

,0
id_vaga,0
titulo,0
id_candidato,0
nome,0
data_candidatura,0
recrutador,0
situacao_candidado,0
target,41783


### Candidatos

In [11]:
with open("applicants.json", "r", encoding="utf-8") as f:
    applicants_raw = json.load(f)

# 2) Normalizar para DataFrame
df_applicants = pd.DataFrame.from_dict(applicants_raw, orient="index")
df_applicants.reset_index(inplace=True)
df_applicants.rename(columns={"index": "id_candidato"}, inplace=True)

# 3) Função para expandir colunas de dicionário
def expand_dict_column(df, col_name):
    if col_name in df.columns:
        df_expanded = pd.json_normalize(df[col_name])
        df_expanded.index = df.index
        df_expanded.columns = [f"{col_name}.{c}" for c in df_expanded.columns]
        return pd.concat([df.drop(columns=[col_name]), df_expanded], axis=1)
    return df

# 4) Expandir colunas aninhadas (incluímos formacao e cargo_atual)
cols_dict = [
    "infos_basicas",
    "informacoes_pessoais",
    "informacoes_profissionais",
    "formacao_e_idiomas",
    "cargo_atual"
]

for col in cols_dict:
    df_applicants = expand_dict_column(df_applicants, col)

# 5) Selecionar colunas úteis (incluindo skills e experiências)
colunas_util = [
    "id_candidato",
    "infos_basicas.objetivo_profissional",
    "informacoes_profissionais.titulo_profissional",
    "informacoes_profissionais.area_atuacao",
    "informacoes_profissionais.conhecimentos_tecnicos",
    "informacoes_profissionais.qualificacoes",
    "informacoes_profissionais.certificacoes",
    "informacoes_profissionais.experiencias",
    "formacao_e_idiomas.nivel_academico",
    "formacao_e_idiomas.nivel_ingles",
    "formacao_e_idiomas.nivel_espanhol",
    "cargo_atual.cargo_atual",
    "informacoes_profissionais.nivel_profissional",
    "formacao_e_idiomas.outro_idioma",
    "formacao_e_idiomas.cursos"

]

applicants = df_applicants[colunas_util].copy()

# 6) Renomear colunas para nomes mais claros
applicants.rename(columns={
    "cv_pt": "cv_texto_pt",
    "infos_basicas.objetivo_profissional": "objetivo_profissional",
    "informacoes_profissionais.titulo_profissional": "titulo_profissional",
    "informacoes_profissionais.area_atuacao": "area_atuacao",
    "informacoes_profissionais.conhecimentos_tecnicos": "conhecimentos_tecnicos",
    "informacoes_profissionais.qualificacoes": "qualificacoes",
    "informacoes_profissionais.certificacoes": "certificacoes",
    "informacoes_profissionais.experiencias": "experiencias",
    "formacao_e_idiomas.nivel_academico": "nivel_academico",
    "formacao_e_idiomas.nivel_ingles": "nivel_ingles",
    "formacao_e_idiomas.nivel_espanhol": "nivel_espanhol",
    "cargo_atual.cargo_atual": "cargo_atual",
    "informacoes_profissionais.nivel_profissional": "nivel_profissional",
    "formacao_e_idiomas.outro_idioma": "outro_idioma",
    "formacao_e_idiomas.cursos": "cursos"
}, inplace=True)

applicants = applicants.replace(r'^\s*$', pd.NA, regex=True)
applicants = applicants.replace(r'-', pd.NA, regex=True)
applicants = applicants.fillna("Não informado")

In [12]:
applicants.head()

,id_candidato,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,31000,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
1,31001,Analista Administrativo,Analista Administrativo,Administrativa,Não informado,Não informado,Não informado,Não informado,Ensino Superior Incompleto,Nenhum,Nenhum,Não informado,Não informado,Não informado,Não informado
2,31002,Administrativo | Financeiro,Administrativo | Financeiro,Administrativa,Não informado,Não informado,Não informado,Não informado,Ensino Superior Completo,Intermediário,Básico,Não informado,Não informado,Não informado,Administração de Empresas
3,31003,Área administrativa,Área administrativa,Administrativa,Não informado,Não informado,Não informado,Não informado,Ensino Superior Incompleto,Nenhum,Nenhum,Não informado,Não informado,Não informado,Não informado
4,31004,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


## Base Final Unificada

Após a leitura, tratamento e padronização das chaves (`id_vaga` e `id_candidato`), realizamos a junção dos três datasets:

- **Jobs (Vagas)** → informações sobre as vagas disponíveis.  
- **Prospects (Situação do candidato na vaga)** → status do processo seletivo (aprovado, recusado, em avaliação, etc).  
- **Applicants (Candidatos)** → dados do candidato: formação, experiência, idiomas, skills, etc.  

A junção foi feita em duas etapas:
1. **Jobs + Prospects** → unificação pelo campo `id_vaga`.  
2. **(Jobs + Prospects) + Applicants** → unificação final pelo campo `id_candidato`.  

### Resultado:
- **DataFrame final:** `df_full`  
- **Linhas:** representa cada candidato em uma vaga, com status e informações pessoais/profissionais.  
- **Colunas:** combinação de atributos da vaga, status do processo e características do candidato.  

Essa base final servirá como **entrada para a etapa de EDA e modelagem preditiva**, onde vamos definir variáveis explicativas (features) e a variável target (status aprovado/recusado).


In [13]:
df_vagas.head(1)

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...


In [14]:
df_prospects.head(1)

,id_vaga,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target
0,4530,CONSULTOR CONTROL M,25632,José Vieira,25-03-2021,Ana Lívia Moreira,Encaminhado ao Requisitante,NaN


In [15]:

df_vagas["id_vaga"] = pd.to_numeric(df_vagas["id_vaga"], errors="coerce").astype("Int64")
df_prospects["id_vaga"] = pd.to_numeric(df_prospects["id_vaga"], errors="coerce").astype("Int64")


df_jobs_prospects = pd.merge(
    df_vagas,
    df_prospects,
    on="id_vaga",
    how="inner"
)

df_jobs_prospects["id_candidato"] = pd.to_numeric(df_jobs_prospects["id_candidato"], errors="coerce").astype("Int64")
applicants["id_candidato"] = pd.to_numeric(applicants["id_candidato"], errors="coerce").astype("Int64")

# Merge
df_full = pd.merge(
    df_jobs_prospects,
    applicants,
    on="id_candidato",
    how="inner"   # só quem existe nas duas bases
)


In [16]:
df_full.head(1)

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,...,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...,...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


In [40]:
df_full.shape

(45071, 31)

In [17]:
df_full.head(1)

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,...,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...,...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


In [18]:
df_full.to_csv("base_completa.csv", index=False)

Leitura do CSV Base Completa

In [95]:
df_final = pd.read_csv("base_completa.csv", sep=',', encoding='utf-8')
df_final.head()
#df_final.count()

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...,Operation Lead -,11010,Dante Nascimento,11-05-2021,Srta. Bella Ferreira,Encaminhado ao Requisitante,NaN,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
1,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26770,Samuel Costa,18-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
2,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26759,Maria Laura Brito,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
3,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26758,Raul Monteiro,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Consultor SAP PP QM,Consultor SAP PP QM,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
4,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26757,José Miguel Cunha,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


Todas as Colunas ou Variáveis do DF

In [96]:
print(df_final.columns.tolist())

['id_vaga', 'inf_titulo_vaga', 'inf_cliente', 'inf_vaga_sap', 'perfil_nivel_academico', 'perfil_nivel profissional', 'perfil_nivel_ingles', 'perfil_nivel_espanhol', 'perfil_competencia_tecnicas_e_comportamentais', 'perfil_principais_atividades', 'titulo', 'id_candidato', 'nome', 'data_candidatura', 'recrutador', 'situacao_candidado', 'target', 'objetivo_profissional', 'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos', 'qualificacoes', 'certificacoes', 'experiencias', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'cargo_atual', 'nivel_profissional', 'outro_idioma', 'cursos']


Verifica o Volume de 0 Reprovados / 1 Aprovados / NaN Não Classificados

In [97]:
df_final["target"].value_counts(dropna=False)

,count
target,
NaN,34961
0.0,7672
1.0,2438


Cria um Dataframe df_aprov_reprov  para usar no modelo de classificação

In [98]:
pd.set_option('display.max_columns', None)
df_aprov_reprov = df_final.dropna(subset=["target"]).copy()
display(df_aprov_reprov.head())#Pega só 0 e 1

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
8,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26570,Isabella Rios,14-05-2021,Srta. Bella Ferreira,Não Aprovado pelo Cliente,0.0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
9,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26139,Matteo Lima,05-05-2021,Eloah Leão,Não Aprovado pelo Cliente,0.0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
23,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,13296,Dra. Isadora da Costa,04-05-2021,Yasmin da Rosa,Não Aprovado pelo Cliente,0.0,Consultor / Arquiteto de Soluções / SAP QM,Consultor / Arquiteto de Soluções / SAP QM,Não informado,Não informado,Não informado,Não informado,Não informado,Mestrado Completo,Fluente,Fluente,Não informado,Não informado,Não informado,Engenharia da Computação
24,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,12658,Mariah Guerra,14-05-2021,Srta. Bella Ferreira,Não Aprovado pelo Cliente,0.0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
26,5183,ANALISTA PL/JR C/ SQL,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Analista,Nenhum,Intermediário,Requisitos mandatórios:\n\no Conhecimentos Téc...,Descrição – Atividades:\n\no Monitoramento das...,ANALISTA PL/JR C/ SQL,26468,Sra. Maria Isis da Paz,04-05-2021,Srta. Maria Carvalho,Não Aprovado pelo RH,0.0,Não informado,Não informado,"Administrativa, Jurídica, Recursos Humanos",Não informado,Não informado,Não informado,Não informado,Pós Graduação Cursando,Básico,Básico,Não informado,Não informado,Não informado,Direito


In [99]:
df_aprov_reprov["target"].value_counts(dropna=False) #Pega só 0 e 1

,count
target,
0.0,7672
1.0,2438


In [100]:
df_aprov_reprov.head()

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
8,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26570,Isabella Rios,14-05-2021,Srta. Bella Ferreira,Não Aprovado pelo Cliente,0.0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
9,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26139,Matteo Lima,05-05-2021,Eloah Leão,Não Aprovado pelo Cliente,0.0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
23,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,13296,Dra. Isadora da Costa,04-05-2021,Yasmin da Rosa,Não Aprovado pelo Cliente,0.0,Consultor / Arquiteto de Soluções / SAP QM,Consultor / Arquiteto de Soluções / SAP QM,Não informado,Não informado,Não informado,Não informado,Não informado,Mestrado Completo,Fluente,Fluente,Não informado,Não informado,Não informado,Engenharia da Computação
24,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,12658,Mariah Guerra,14-05-2021,Srta. Bella Ferreira,Não Aprovado pelo Cliente,0.0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
26,5183,ANALISTA PL/JR C/ SQL,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Analista,Nenhum,Intermediário,Requisitos mandatórios:\n\no Conhecimentos Téc...,Descrição – Atividades:\n\no Monitoramento das...,ANALISTA PL/JR C/ SQL,26468,Sra. Maria Isis da Paz,04-05-2021,Srta. Maria Carvalho,Não Aprovado pelo RH,0.0,Não informado,Não informado,"Administrativa, Jurídica, Recursos Humanos",Não informado,Não informado,Não informado,Não informado,Pós Graduação Cursando,Básico,Básico,Não informado,Não informado,Não informado,Direito


In [30]:
df_aprov_reprov.to_csv("aprovados_reprovados.csv", index=False)

Cria um Dataframe df_n_classificados para usar como input no Streamlit

In [74]:
pd.set_option('display.max_columns', None)
df_n_classificados = df_final[df_final["target"].isna()].copy()
display(df_n_classificados.head())#Pega só NaN

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...,Operation Lead -,11010,Dante Nascimento,11-05-2021,Srta. Bella Ferreira,Encaminhado ao Requisitante,NaN,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
1,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26770,Samuel Costa,18-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
2,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26759,Maria Laura Brito,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
3,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26758,Raul Monteiro,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Consultor SAP PP QM,Consultor SAP PP QM,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
4,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26757,José Miguel Cunha,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


In [41]:
# nan_counts_n_classificados = df_n_classificados.isnull().sum()
# print(nan_counts_n_classificados)

In [43]:
df_n_classificados.to_csv("nao_classificados.csv", index=False)

Modelo

In [92]:
import numpy as np

In [101]:
map_nivel = {
    "Nenhum": 0,
    "Básico": 1,
    "Intermediário": 2,
    "Avançado": 3,
    "Fluente": 4,
    "Não informado": 0
}

col_niveis = ["perfil_nivel_ingles"
, "perfil_nivel_espanhol", "nivel_ingles", "nivel_espanhol"]

for col in col_niveis:
    df_aprov_reprov[col] = df_aprov_reprov[col].map(map_nivel).fillna(0).astype(int)
#

In [102]:
col_textuais = ["perfil_competencia_tecnicas_e_comportamentais",
                "perfil_principais_atividades",
                "titulo_profissional",
                "area_atuacao",
                "conhecimentos_tecnicos",
                "qualificacoes",
                "certificacoes",
                "cargo_atual",
                "nivel_profissional",
                "outro_idioma",
                "cursos"]

for col in col_textuais:
    df_aprov_reprov[col] = np.where((df_aprov_reprov[col].str.lower().str.contains("não informado")) | # Added the missing '|'
                                  (df_aprov_reprov[col].isna()) |
                                  (df_aprov_reprov[col].str.strip() == ""), 0, 1)

In [103]:
df_aprov_reprov.head()

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
8,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,4,0,1,1,Consultor PP/QM Sênior,26570,Isabella Rios,14-05-2021,Srta. Bella Ferreira,Não Aprovado pelo Cliente,0.0,Não informado,0,0,0,0,0,Não informado,Não informado,0,0,0,0,0,0
9,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,4,0,1,1,Consultor PP/QM Sênior,26139,Matteo Lima,05-05-2021,Eloah Leão,Não Aprovado pelo Cliente,0.0,Não informado,0,0,0,0,0,Não informado,Não informado,0,0,0,0,0,0
23,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,4,0,1,1,Consultor PP/QM Sênior,13296,Dra. Isadora da Costa,04-05-2021,Yasmin da Rosa,Não Aprovado pelo Cliente,0.0,Consultor / Arquiteto de Soluções / SAP QM,1,0,0,0,0,Não informado,Mestrado Completo,4,4,0,0,0,1
24,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,4,0,1,1,Consultor PP/QM Sênior,12658,Mariah Guerra,14-05-2021,Srta. Bella Ferreira,Não Aprovado pelo Cliente,0.0,Não informado,0,0,0,0,0,Não informado,Não informado,0,0,0,0,0,0
26,5183,ANALISTA PL/JR C/ SQL,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Analista,0,2,1,1,ANALISTA PL/JR C/ SQL,26468,Sra. Maria Isis da Paz,04-05-2021,Srta. Maria Carvalho,Não Aprovado pelo RH,0.0,Não informado,0,1,0,0,0,Não informado,Pós Graduação Cursando,1,1,0,0,0,1


In [104]:
X = df_aprov_reprov[col_niveis + col_textuais]
y = df_aprov_reprov["target"].astype(int)


In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [107]:
model = RandomForestClassifier(n_estimators=200,
                               max_depth=10,
                               random_state=42,
                               class_weight="balanced")
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=200,
                       random_state=42)

In [110]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6894164193867458
Precision: 0.3909657320872274
F1 Score: 0.44424778761061945
Recall: 0.514344262295082

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.75      0.78      1534
           1       0.39      0.51      0.44       488

    accuracy                           0.69      2022
   macro avg       0.61      0.63      0.61      2022
weighted avg       0.72      0.69      0.70      2022



In [111]:
probs = model.predict_proba(X_test)[:, 1]

In [124]:
colunas_info = ["id_vaga",
                "inf_titulo_vaga",
                "inf_cliente",
                "perfil_nivel_academico",
                "perfil_nivel profissional",
                "nome",
                "data_candidatura",
                "recrutador",
                "situacao_candidado",
                "target"

                ]
ranking = X_test.copy()
ranking["prob_aprovado"] = probs
ranking["real"] = y_test.values

In [126]:
ranking = ranking.merge(df_aprov_reprov[colunas_info],
                        left_index=True,
                        right_index=True,  # Merge on index
                        how="left"
                        )

In [127]:
ranking = ranking.sort_values(by="prob_aprovado", ascending=False)

In [128]:
print("\nRanking dos 10 melhores candidatos:\n")
ranking.head(10)


Ranking dos 10 melhores candidatos:



,perfil_nivel_ingles,perfil_nivel_espanhol,nivel_ingles,nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,cargo_atual,nivel_profissional,outro_idioma,cursos,prob_aprovado,real,id_vaga_x,inf_titulo_vaga_x,inf_cliente_x,perfil_nivel_academico_x,perfil_nivel profissional_x,nome_x,data_candidatura_x,recrutador_x,situacao_candidado_x,target_x,id_vaga_y,inf_titulo_vaga_y,inf_cliente_y,perfil_nivel_academico_y,perfil_nivel profissional_y,nome_y,data_candidatura_y,recrutador_y,situacao_candidado_y,target_y
42297,3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.945155,1,528,MSC - Analista de Testes,Miller-Curry,Ensino Superior Incompleto,Analista,Sr. Apollo da Costa,01-04-2019,Carlos Eduardo Ribeiro,Contratado pela Decision,1.0,528,MSC - Analista de Testes,Miller-Curry,Ensino Superior Incompleto,Analista,Sr. Apollo da Costa,01-04-2019,Carlos Eduardo Ribeiro,Contratado pela Decision,1.0
41891,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.941651,1,681,Estoque,Miller-Curry,Ensino Médio Completo,Sênior,Dr. Guilherme Duarte,30-04-2019,Dra. Sophie Azevedo,Aprovado,1.0,681,Estoque,Miller-Curry,Ensino Médio Completo,Sênior,Dr. Guilherme Duarte,30-04-2019,Dra. Sophie Azevedo,Aprovado,1.0
42216,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0.921402,1,566,Assistente Adm. Junior (ITAU),Miller-Curry,Ensino Médio Completo,Assistente,Benjamim da Conceição,10-04-2019,Paulo Peixoto,Contratado pela Decision,1.0,566,Assistente Adm. Junior (ITAU),Miller-Curry,Ensino Médio Completo,Assistente,Benjamim da Conceição,10-04-2019,Paulo Peixoto,Contratado pela Decision,1.0
42217,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0.921402,1,566,Assistente Adm. Junior (ITAU),Miller-Curry,Ensino Médio Completo,Assistente,Pedro Henrique Rocha,10-04-2019,Paulo Peixoto,Contratado pela Decision,1.0,566,Assistente Adm. Junior (ITAU),Miller-Curry,Ensino Médio Completo,Assistente,Pedro Henrique Rocha,10-04-2019,Paulo Peixoto,Contratado pela Decision,1.0
42215,1,2,0,0,1,1,0,0,0,0,0,1,0,0,0,0.892563,1,567,Assist. Adm. Pleno (DOW),Miller-Curry,Ensino Superior Cursando,Assistente,Srta. Maria Cirino,03-04-2019,Dra. Sophie Azevedo,Contratado pela Decision,1.0,567,Assist. Adm. Pleno (DOW),Miller-Curry,Ensino Superior Cursando,Assistente,Srta. Maria Cirino,03-04-2019,Dra. Sophie Azevedo,Contratado pela Decision,1.0
28571,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0.805479,1,1897,PROXXI JR,Miller-Curry,Ensino Médio Completo,Júnior,Raul Novaes,27-12-2019,Dr. Luan Sá,Contratado pela Decision,1.0,1897,PROXXI JR,Miller-Curry,Ensino Médio Completo,Júnior,Raul Novaes,27-12-2019,Dr. Luan Sá,Contratado pela Decision,1.0
28569,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0.805479,1,1899,PROXXI JR,Miller-Curry,Ensino Médio Completo,Júnior,Ravi Araújo,27-12-2019,Dr. Luan Sá,Contratado pela Decision,1.0,1899,PROXXI JR,Miller-Curry,Ensino Médio Completo,Júnior,Ravi Araújo,27-12-2019,Dr. Luan Sá,Contratado pela Decision,1.0
28752,0,0,1,1,1,1,1,0,1,0,0,1,0,0,1,0.755815,1,1795,Analista de Testes,Gonzalez and Sons,Ensino Médio Completo,Sênior,Dr. Gabriel Fonseca,03-12-2019,Liz Freitas,Contratado pela Decision,1.0,1795,Analista de Testes,Gonzalez and Sons,Ensino Médio Completo,Sênior,Dr. Gabriel Fonseca,03-12-2019,Liz Freitas,Contratado pela Decision,1.0
28143,0,0,1,4,1,1,1,1,0,0,0,0,0,0,0,0.747791,0,2098,Analista de Faturamento,Jimenez PLC,Ensino Superior Cursando,Analista,Eduarda Vasconcelos,03-02-2020,Liz Freitas,Não Aprovado pelo Requisitante,0.0,2098,Analista de Faturamento,Jimenez PLC,Ensino Superior Cursando,Analista,Eduarda Vasconcelos,03-02-2020,Liz Freitas,Não Aprovado pelo Requisitante,0.0
29681,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,0.726789,1,1379,Assistente de Estoque Senior,Miller-Curry,Ensino Superior Completo,Sênior,Gael Melo,04-09-2019,Benjamin Castro,Aprovado,1.0,1379,Assistente de Estoque Senior,Miller-Curry,Ensino Superior Completo,Sênior,Gael Melo,04-09-2019,Benjamin Castro,Aprovado,1.0


NOVA BASE

In [140]:
df_novos = pd.read_csv("nao_classificados.csv", sep=',', encoding='utf-8')
df_novos.head()

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5185,Operation Lead -,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Avançado,Fluente,Required Skills:\n• Prior experience in Cloud ...,Operations Lead\n\nRoles & Responsibilities:\n...,Operation Lead -,11010,Dante Nascimento,11-05-2021,Srta. Bella Ferreira,Encaminhado ao Requisitante,NaN,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,CONSULTOR BASIS ECC/ S/4HANA / Solution Manage...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
1,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26770,Samuel Costa,18-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
2,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26759,Maria Laura Brito,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
3,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26758,Raul Monteiro,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Consultor SAP PP QM,Consultor SAP PP QM,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
4,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em pr...,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,Consultor PP/QM Sênior,26757,José Miguel Cunha,17-05-2021,Srta. Bella Ferreira,Prospect,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado


In [141]:
for col in col_niveis:
    df_novos[col] = df_novos[col].map(map_nivel).fillna(0).astype(int)


for col in col_textuais:
  df_novos[col] = np.where((df_novos[col].str.lower().str.contains("não informado")) | # Added the missing '|'
                                  (df_novos[col].isna()) |
                                  (df_novos[col].str.strip() == ""), 0, 1)

In [142]:
X_novos =df_novos[col_niveis + col_textuais]

In [143]:
probs_novos = model.predict_proba(X_novos)[:, 1]

ranking_novos = X_novos.copy()
ranking_novos["prob_aprovado"] = probs_novos

colunas_info = ["id_vaga",
                "inf_titulo_vaga",
                "inf_cliente",
                "perfil_nivel_academico",
                "perfil_nivel profissional",
                "nome",
                "data_candidatura",
                "recrutador",
                "situacao_candidado",
                "target"

                ]

ranking_novos = ranking_novos.merge(df_novos[colunas_info],
                        left_index=True,
                        right_index=True,  # Merge on index
                        how="left"
                        )

In [144]:
ranking_novos = ranking_novos.sort_values(by="prob_aprovado", ascending=False)

In [145]:
ranking_novos.head(1000)

,perfil_nivel_ingles,perfil_nivel_espanhol,nivel_ingles,nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,cargo_atual,nivel_profissional,outro_idioma,cursos,prob_aprovado,id_vaga,inf_titulo_vaga,inf_cliente,perfil_nivel_academico,perfil_nivel profissional,nome,data_candidatura,recrutador,situacao_candidado,target
32580,4,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.962762,564,Web Developer Front-End,Miller-Curry,Ensino Superior Completo,Pleno,Ana Cecília Silveira,13-05-2019,Paulo Peixoto,Prospect,NaN
23894,3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.945155,433,3659759 - Mainframe,Miller-Curry,Ensino Superior Cursando,Júnior,Samuel Pimenta,18-03-2019,Carlos Eduardo Ribeiro,Inscrito,NaN
32626,3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.945155,528,MSC - Analista de Testes,Miller-Curry,Ensino Superior Incompleto,Analista,Dr. Lorenzo Montenegro,03-04-2019,Carlos Eduardo Ribeiro,Prospect,NaN
5244,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,11092,Analista Adm. Junior - Latam - Compras - Repla...,Miller-Curry,Ensino Médio Completo,Analista,Dr. Apollo Siqueira,09-03-2023,Caroline Machado,Inscrito,NaN
15163,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,7794,Associate Operations - Vale (12562096),Nelson-Page,Ensino Médio Completo,Assistente,Sr. Erick Vargas,16-12-2021,Laura Pacheco,Encaminhado ao Requisitante,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.686075,5959,Recrutador TI,Barnes-Woods,Ensino Médio Completo,Sênior,Sarah Albuquerque,13-07-2021,Srta. Maria Carvalho,Prospect,NaN
2686,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.686075,5959,Recrutador TI,Barnes-Woods,Ensino Médio Completo,Sênior,Isabella Barbosa,13-07-2021,Srta. Maria Carvalho,Prospect,NaN
26557,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.686075,4245,Desenvolvedor C#/.net – Senior,Callahan-Hall,Ensino Médio Completo,Pleno,Nina da Luz,15-02-2021,Carolina Araújo,Encaminhado ao Requisitante,NaN
26558,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.686075,4244,Analista de testes SAP,"Morris, Moran and Dodson",Ensino Superior Completo,Sênior,Srta. Giovanna Castro,24-02-2021,Srta. Bella Ferreira,Encaminhado ao Requisitante,NaN


In [146]:
display(ranking_novos.head(10))

,perfil_nivel_ingles,perfil_nivel_espanhol,nivel_ingles,nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,cargo_atual,nivel_profissional,outro_idioma,cursos,prob_aprovado,id_vaga,inf_titulo_vaga,inf_cliente,perfil_nivel_academico,perfil_nivel profissional,nome,data_candidatura,recrutador,situacao_candidado,target
32580,4,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.962762,564,Web Developer Front-End,Miller-Curry,Ensino Superior Completo,Pleno,Ana Cecília Silveira,13-05-2019,Paulo Peixoto,Prospect,NaN
23894,3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.945155,433,3659759 - Mainframe,Miller-Curry,Ensino Superior Cursando,Júnior,Samuel Pimenta,18-03-2019,Carlos Eduardo Ribeiro,Inscrito,NaN
32626,3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.945155,528,MSC - Analista de Testes,Miller-Curry,Ensino Superior Incompleto,Analista,Dr. Lorenzo Montenegro,03-04-2019,Carlos Eduardo Ribeiro,Prospect,NaN
5244,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,11092,Analista Adm. Junior - Latam - Compras - Repla...,Miller-Curry,Ensino Médio Completo,Analista,Dr. Apollo Siqueira,09-03-2023,Caroline Machado,Inscrito,NaN
15163,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,7794,Associate Operations - Vale (12562096),Nelson-Page,Ensino Médio Completo,Assistente,Sr. Erick Vargas,16-12-2021,Laura Pacheco,Encaminhado ao Requisitante,NaN
30129,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,8855,Assistente Administrativo/ Controlador de Acesso.,"Morris, Moran and Dodson",Ensino Médio Completo,Assistente,Srta. Mariah Siqueira,09-04-2022,Caroline Machado,Inscrito,NaN
30153,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,8854,Assistente Administrativo / Controlador de Acesso,"Morris, Moran and Dodson",Ensino Médio Completo,Auxiliar,Henrique Nogueira,03-04-2022,Caroline Machado,Inscrito,NaN
29988,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,8861,Assistente Administrativo / Controlador de Ace...,"Morris, Moran and Dodson",Ensino Médio Completo,Assistente,Rafaela Borges,20-04-2022,Caroline Machado,Inscrito,NaN
30165,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,8854,Assistente Administrativo / Controlador de Acesso,"Morris, Moran and Dodson",Ensino Médio Completo,Auxiliar,Ana Vitória Araújo,31-03-2022,Caroline Machado,Inscrito,NaN
30071,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,8858,Assistente Administrativo / Controlador de Acesso,"Morris, Moran and Dodson",Ensino Médio Completo,Assistente,Ana Julia Ramos,31-03-2022,Caroline Machado,Inscrito,NaN


In [151]:
# Ensure id_vaga is of the same type in both DataFrames before merging
ranking_novos['id_vaga'] = pd.to_numeric(ranking_novos['id_vaga'], errors='coerce').astype('Int64')
df_vagas['id_vaga'] = pd.to_numeric(df_vagas['id_vaga'], errors='coerce').astype('Int64')


# Merge ranking_novos with df_vagas on id_vaga
candidates_with_vacancy_details = ranking_novos.merge(
    df_vagas,
    on='id_vaga',
    how='left'  # Use a left merge to keep all candidates from ranking_novos
)

# Define columns to keep
columns_to_keep = [col for col in candidates_with_vacancy_details.columns if col not in ['perfil_competencia_tecnicas_e_comportamentais_y', 'perfil_principais_atividades_y']]

# Display the head of the merged DataFrame with selected columns
display(candidates_with_vacancy_details[columns_to_keep].head())

,perfil_nivel_ingles_x,perfil_nivel_espanhol_x,nivel_ingles,nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais_x,perfil_principais_atividades_x,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,cargo_atual,nivel_profissional,outro_idioma,cursos,prob_aprovado,id_vaga,inf_titulo_vaga_x,inf_cliente_x,perfil_nivel_academico_x,perfil_nivel profissional_x,nome,data_candidatura,recrutador,situacao_candidado,target,inf_titulo_vaga_y,inf_cliente_y,inf_vaga_sap,perfil_nivel_academico_y,perfil_nivel profissional_y,perfil_nivel_ingles_y,perfil_nivel_espanhol_y
0,4,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.962762,564,Web Developer Front-End,Miller-Curry,Ensino Superior Completo,Pleno,Ana Cecília Silveira,13-05-2019,Paulo Peixoto,Prospect,NaN,Web Developer Front-End,Miller-Curry,Não,Ensino Superior Completo,Pleno,Fluente,Nenhum
1,3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.945155,433,3659759 - Mainframe,Miller-Curry,Ensino Superior Cursando,Júnior,Samuel Pimenta,18-03-2019,Carlos Eduardo Ribeiro,Inscrito,NaN,3659759 - Mainframe,Miller-Curry,Não,Ensino Superior Cursando,Júnior,Avançado,Nenhum
2,3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.945155,528,MSC - Analista de Testes,Miller-Curry,Ensino Superior Incompleto,Analista,Dr. Lorenzo Montenegro,03-04-2019,Carlos Eduardo Ribeiro,Prospect,NaN,MSC - Analista de Testes,Miller-Curry,Não,Ensino Superior Incompleto,Analista,Avançado,Nenhum
3,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,11092,Analista Adm. Junior - Latam - Compras - Repla...,Miller-Curry,Ensino Médio Completo,Analista,Dr. Apollo Siqueira,09-03-2023,Caroline Machado,Inscrito,NaN,Analista Adm. Junior - Latam - Compras - Repla...,Miller-Curry,Não,Ensino Médio Completo,Analista,Nenhum,Nenhum
4,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0.878517,7794,Associate Operations - Vale (12562096),Nelson-Page,Ensino Médio Completo,Assistente,Sr. Erick Vargas,16-12-2021,Laura Pacheco,Encaminhado ao Requisitante,NaN,Associate Operations - Vale (12562096),Nelson-Page,Não,Ensino Médio Completo,Assistente,Nenhum,Nenhum
